In [180]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np

In [181]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 29/12/22 - Horário: 11:38:16.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:06.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:10.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:20.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 29/12/22 - Horário: 11:42:20.  Portanto não foi baixado novamente.
vrapeelcronograna já foi baixado hoje: Dia: 29/12/22 - Horário: 11:43:35.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 29/12/22 - Horário: 11:43:48.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 29/12/22 - Horário: 11:44:27.  Portanto não foi baixado novamente.
vrapeelo

In [182]:
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao = monitoramentoleilao.loc[monitoramentoleilao.groupby('IdeUsinaOutorga').DatInicioSuprimento.idxmin()]

In [183]:
monitoramentodados  = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))
io_rapeel = pd.read_parquet(os.path.join(directory,"vrapeelcronograna.gzip"))[['DthEnvio','IdeUsinaOutorga','DatPrevistaAprovacaoIII','DatRealizacaoIII']].drop_duplicates(inplace=True)

In [184]:
rename_cols = {'DthEnvio' : 'Data_Envio',
'NomUsina' : 'usina_nome',
'DatRealizacaoII' : 'canteiroReal',
'DatPrevistaAprovacaoIII' : 'prev_IO',
'DatRealizacaoIII' : 'IO_real',
'DatRealizacaoIX' : 'DesvRio_real',
'DatRealizacaoXI' : 'Ench_Real',
'DatRealizacaoXIII' : 'IOTrans_Real',
'DatRealizacaoXIV' : 'Conc_Trans_REAL',
'DatRealizacaoXII' : 'Comiss_Real',
'DatRealizacaoVII' : 'ME_Real_conc_eol',
'DatRealizacaoVIII' : 'Conc_Combust_Real',}
cols = ['IdeUsinaOutorga','CodCeg','NomUsina','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV',]
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograna.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [185]:
rename_cols = {
    'DatPrevistaComercial' : 'DatPrevistaComercial1'
}
cols = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','SigTipoGeracao','DscObservacao','VlrPotenciaUnitaria','NumOperacaoUg','DatPrevistaComercial','DatPrevistaRotor','DatPrevistaMaximaRotor','DatRealizacaoRotor','DatPrevistaTeste','DatPrevistaMaximaTeste','DatRealizacaoTeste','DatPrevistaComercial','DatPrevistaMaximaComercial','DatRealizacaoComercial','SigTipoGeracao','NomUsina']
ugrapeel  = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [186]:
rename_cols = {
'DthEnvio' : 'Data_Envio',
'SigTipoGeracao' : 'TipoGeracao',
'NomUsina' : 'usina_nome',
'NumOperacaoUg' : 'Num_UG',
'VlrPotenciaUnitaria' : 'Pot_UG',
'DatRealizacaoRotor' : 'UG_decidarotor_real',
'DatPrevistaTeste' : 'UG_op_test_previsão',
'DatRealizacaoTeste' : 'UG_op_test_real',
'DatPrevistaComercial' : 'UG_op_com_previsão',
'DatRealizacaoComercial' : 'UG_op_com_real',
}

cols = ['DthEnvio','IdeUsinaOutorga','SigTipoGeracao','NomUsina','NumOperacaoUg','VlrPotenciaUnitaria','DatRealizacaoRotor','DatPrevistaTeste','DatRealizacaoTeste','DatPrevistaComercial','DatRealizacaoComercial','DscObservacao','DthEnvio','IdeUsinaOutorga','NumOperacaoUg']
inputug = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [187]:
cols = ['IdeUsinaOutorga','NumUgUsina','MdaPotenciaUnitaria','MdaPotenciaLiberadaTeste','MdaPotenciaLiberadaComercial','NumDespachoTeste','numDespachoComercial','IdcMonitorada','DatDescidaoutorgadaRotor','DatRealizacaoSFGRotor','DatInicioOpTesteOutorgada','DatLiberacaoSFGTeste','DatUGInicioOpComerOutorgado','DatLiberOpComerRealizado','DatPrevisaoSFGComercial']
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols].drop_duplicates()

In [188]:
cols = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','PctAoACR','PctAoACL','PctAutoproducao','NomSituacaoI','PctEnergiaContratoAssinadoI','DatConclusaoI','NomSituacaoII','PctEnergiaContratoAssinadoII','DatConclusaoII','NomSituacaoIII','PctEnergiaContratoAssinadoIII','DatConclusaoIII','DscObservacao']
rename_cols = {'NomSituacaoI' : 'SituacaoACR',
'PctEnergiaContratoAssinadoI' : 'PctEnergiaContratoAssinadoACR',
'DatConclusaoI' : 'DatConclusaoACR',
'NomSituacaoII' : 'SituacaoCCESI',
'PctEnergiaContratoAssinadoII' : 'PctEnergiaContratoAssinadoCCESI',
'DatConclusaoII' : 'DatConclusaoCCESI',
'NomSituacaoIII' : 'SituacaoACL',
'PctEnergiaContratoAssinadoIII' : 'PctEnergiaContratoAssinadoACL',
'DatConclusaoIII' : 'DatConclusaoACL',
'DscObservacao' : 'DscObservacaomercado'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [189]:
cols= ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','DatSolicitacaoI','DatPrevistaEmissaoI','DatEmissaoI','DatValidadeI','NomSituacaoI','DatSolicitacaoIV','DatPrevistaEmissaoIV','DatEmissaoIV','DatValidadeIV','NomSituacaoIV','DatSolicitacaoV','DatPrevistaEmissaoV','DatEmissaoV','DatValidadeV','NomSituacaoV','IdcTipoEntrave','DscSolucao','DscObservacao']
rename_cols = {
'DatSolicitacaoI' : 'DatSolicitacaoLP',
'DatPrevistaEmissaoI' : 'DatPrevistaEmissaoLP',
'DatEmissaoI' : 'DatEmissaoLP',
'DatValidadeI' : 'DatValidadeLP',
'NomSituacaoI' : 'NomSituacaoLP',
'DatSolicitacaoIV' : 'DatSolicitacaoLI',
'DatPrevistaEmissaoIV' : 'DatPrevistaEmissaoLI',
'DatEmissaoIV' : 'DatEmissaoLI',
'DatValidadeIV' : 'DatValidadeLI',
'NomSituacaoIV' : 'NomSituacaoLI',
'DatSolicitacaoV' : 'DatSolicitacaoLO',
'DatPrevistaEmissaoV' : 'DatPrevistaEmissaoLO',
'DatEmissaoV' : 'DatEmissaoLO',
'DatValidadeV' : 'DatValidadeLO',
'NomSituacaoV' : 'NomSituacaoLO',
'DscObservacao' : 'DscObservacaoAmbiental'}
ambientaldatas =  pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [190]:
cols = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','IdcTipoInstalacao','IdcNecessidade','NomSituacaoParAcesso','DatEmissaoParAcesso','NomSitContratoI','DatAssinaturaContratoI','DatValidadeContratoI','NomSitContratoII','DatAssinaturaContratoII','DatValidadeContratoII','NomSituacaoContratoIII','DatAssinaturaContratoIII','DatValidadeContratoIII','NomSitContratoIV','DatAssinaturaContratoIV','DatValidadeContratoIV','DscObservacao']
rename_cols = {'NomSitContratoI' : 'NomSitContratoCCD',
'DatAssinaturaContratoI' : 'DatAssinaturaContratoCCD',
'DatValidadeContratoI' : 'DatValidadeContratoCCD',
'NomSitContratoII' : 'NomSitContratoCCT',
'DatAssinaturaContratoII' : 'DatAssinaturaContratoCCT',
'DatValidadeContratoII' : 'DatValidadeContratoCCT',
'NomSituacaoContratoIII' : 'NomSituacaoContratoCUSD',
'DatAssinaturaContratoIII' : 'DatAssinaturaContratoCUSD',
'DatValidadeContratoIII' : 'DatValidadeContratoCUSD',
'NomSitContratoIV' : 'NomSitContratoCUST',
'DatAssinaturaContratoIV' : 'DatAssinaturaContratoCUST',
'DatValidadeContratoIV' : 'DatValidadeContratoCUST',
'DscObservacao' : 'DscObservacaoAcesso'}
acessocontratos =  pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [191]:
cols1 = ['IdeUsinaOutorga','DthEnvio','CodCeg','NomUsina','PctAoACR','PctAoACL','PctAutoproducao','SituacaoACR','PctEnergiaContratoAssinadoACR','DatConclusaoACR','SituacaoCCESI','PctEnergiaContratoAssinadoCCESI','DatConclusaoCCESI','SituacaoACL','PctEnergiaContratoAssinadoACL','DatConclusaoACL','DscObservacaomercado']
cols2 = ['IdeUsinaOutorga','DthEnvio','DatSolicitacaoLP','DatPrevistaEmissaoLP','DatEmissaoLP','DatValidadeLP','NomSituacaoLP','DatSolicitacaoLI','DatPrevistaEmissaoLI','DatEmissaoLI','DatValidadeLI','NomSituacaoLI','DatSolicitacaoLO','DatPrevistaEmissaoLO','DatEmissaoLO','DatValidadeLO','NomSituacaoLO','IdcTipoEntrave','DscSolucao','DscObservacaoAmbiental']
cols3 = ["IdeUsinaOutorga","DthEnvio",'IdcTipoInstalacao','IdcNecessidade','NomSituacaoParAcesso','DatEmissaoParAcesso','NomSitContratoCCD','DatAssinaturaContratoCCD','DatValidadeContratoCCD','NomSitContratoCCT','DatAssinaturaContratoCCT','DatValidadeContratoCCT','NomSituacaoContratoCUSD','DatAssinaturaContratoCUSD','DatValidadeContratoCUSD','NomSitContratoCUST','DatAssinaturaContratoCUST','DatValidadeContratoCUST','DscObservacaoAcesso', ]
inputrapeelv = pd.merge(mercado[cols1],ambientaldatas[cols2],on=["IdeUsinaOutorga","DthEnvio"],how="inner")
inputrapeelv = pd.merge(inputrapeelv,acessocontratos[cols3],on=["IdeUsinaOutorga","DthEnvio"],how="inner")

In [192]:
rapeelatualizado = inputrapeelv.loc[:,['IdeUsinaOutorga','DthEnvio']]
rapeelatualizado =rapeelatualizado.loc[rapeelatualizado.groupby("IdeUsinaOutorga").DthEnvio.idxmax()].rename(columns={'DthEnvio': 'DthEnvioMAX'})

In [193]:
cols1 = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','PctAoACR','PctAoACL','PctAutoproducao','SituacaoACR','PctEnergiaContratoAssinadoACR','DatConclusaoACR','SituacaoCCESI','PctEnergiaContratoAssinadoCCESI','DatConclusaoCCESI','SituacaoACL','PctEnergiaContratoAssinadoACL','DatConclusaoACL','IdcTipoInstalacao','IdcNecessidade','NomSituacaoParAcesso','DatEmissaoParAcesso','NomSitContratoCCD','DatAssinaturaContratoCCD','DatValidadeContratoCCD','NomSitContratoCCT','DatAssinaturaContratoCCT','DatValidadeContratoCCT','NomSituacaoContratoCUSD','DatAssinaturaContratoCUSD','DatValidadeContratoCUSD','NomSitContratoCUST','DatAssinaturaContratoCUST','DatValidadeContratoCUST','DatSolicitacaoLP','DatPrevistaEmissaoLP','DatEmissaoLP','DatValidadeLP','NomSituacaoLP','DatSolicitacaoLI','DatPrevistaEmissaoLI','DatEmissaoLI','DatValidadeLI','NomSituacaoLI','DatSolicitacaoLO','DatPrevistaEmissaoLO','DatEmissaoLO','DatValidadeLO','NomSituacaoLO','IdcTipoEntrave','DscSolucao','DscObservacaoAcesso','DscObservacaomercado','DscObservacaoAmbiental']
cols2 = ['IdeUsinaOutorga','DthEnvioMAX']
cols3 = ['IdeUsinaOutorga','DscObservacao','DthEnvio']
definicoes = pd.merge(inputrapeelv[cols1],rapeelatualizado[cols2],left_on=['IdeUsinaOutorga','DthEnvio'],right_on=['IdeUsinaOutorga','DthEnvioMAX'],how="inner")
definicoes = pd.merge(definicoes,ugrapeel[cols3],on=['IdeUsinaOutorga','DthEnvio'],how="inner").drop_duplicates().rename(columns={
    'DscObservacao' : 'DscObservacaoUG'
})

dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

definicoes['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    definicoes.loc[definicoes.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = definicoes[validade]

In [194]:
definicoesmonitoramento = pd.merge(monitoramentodados,definicoes,on="IdeUsinaOutorga",how="left")

definicoesmonitoramento['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    definicoesmonitoramento.loc[definicoesmonitoramento[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]
definicoesmonitoramento.loc[definicoesmonitoramento.CondicaoAmbiental.isna(),'CondicaoAmbiental'] = 'Sem LP'
definicoesmonitoramento[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental,CondicaoAmbiental
1997,NaT,2027-05-28,2025-04-15,2027-05-28,LI
1269,NaT,NaT,NaT,NaT,Sem LP
854,NaT,NaT,2024-01-03,2024-01-03,LP
2964,2025-12-31,2025-12-31,2025-12-31,2025-12-31,LO
2349,NaT,NaT,2023-03-22,2023-03-22,LP
767,NaT,NaT,2029-11-02,2029-11-02,LP
2342,2031-02-24,2031-02-24,2031-02-24,2031-02-24,LO
1112,NaT,2023-07-05,2024-09-05,2023-07-05,LI
2235,NaT,2024-06-29,2023-02-22,2024-06-29,LI
692,2028-03-08,2023-03-09,2019-12-18,2028-03-08,LO


In [195]:
definicoesmonitoramento['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

definicoesmonitoramento.loc[
    ((definicoesmonitoramento.NomSitContratoCCD ==  nsa)&
    (definicoesmonitoramento.NomSitContratoCCT == nsa) &
    definicoesmonitoramento.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

definicoesmonitoramento.loc[
(    ((definicoesmonitoramento.NomSitContratoCCD ==  valido) |
    (definicoesmonitoramento.NomSitContratoCCT == valido)) &
    definicoesmonitoramento.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.NomSitContratoCCD ==  vencido) |
    (definicoesmonitoramento.NomSitContratoCCT == vencido) |
    (definicoesmonitoramento.NomSitContratoCCD == nass) |
    (definicoesmonitoramento.NomSitContratoCCT == nass )) &
    definicoesmonitoramento.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(definicoesmonitoramento[['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


definicoesmonitoramento['CondicaoUso'] = "Não informado"

definicoesmonitoramento.loc[
    ((definicoesmonitoramento.NomSituacaoContratoCUSD ==  nsa)&
    (definicoesmonitoramento.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


definicoesmonitoramento.loc[
    ((definicoesmonitoramento.NomSituacaoContratoCUSD ==  valido) |
    (definicoesmonitoramento.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.NomSituacaoContratoCUSD ==  vencido) |
    (definicoesmonitoramento.NomSitContratoCUST == vencido) |
    (definicoesmonitoramento.NomSituacaoContratoCUSD == nass) |
    (definicoesmonitoramento.NomSitContratoCUST == nass )) &
    definicoesmonitoramento.DthEnvio.notna())
    ,'CondicaoUso'] = "Sem Uso"

display(definicoesmonitoramento[['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


definicoesmonitoramento["PPA"] = "Ambos"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.DscComercializacaoEnergia == "ACR") & 
        definicoesmonitoramento.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoesmonitoramento.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoesmonitoramento.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(definicoesmonitoramento[['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
714,Não se Aplica,Válido,2022-12-05 19:25:51.040,OK
1485,Não se Aplica,Não Assinado,2022-12-01 21:22:42.267,Sem Conexão
1664,Não se Aplica,Não Assinado,2022-12-02 14:42:37.473,Sem Conexão
2266,Não se Aplica,Não Assinado,2022-12-01 09:50:07.697,Sem Conexão
416,Não se Aplica,Não Assinado,2022-12-06 12:03:11.473,Sem Conexão
524,NaN,NaN,NaT,Não informado
331,Não se Aplica,Válido,2022-01-05 16:57:28.327,OK
343,Não se Aplica,Válido,2022-12-05 20:26:16.817,OK
2728,NaN,NaN,NaT,Não informado
2200,NaN,NaN,NaT,Não informado


,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
97,Não se Aplica,Válido,OK
2703,Não se Aplica,Não Assinado,Sem Conexão
1792,Não se Aplica,Válido,OK
9,Não Assinado,Não se Aplica,Sem Conexão
2533,Não se Aplica,Não se Aplica,Verificar
732,Não se Aplica,Válido,OK
1272,NaN,NaN,Não informado
2778,Não se Aplica,Não Assinado,Sem Conexão
693,Não se Aplica,Não se Aplica,Verificar
708,Não se Aplica,Válido,OK


,DscComercializacaoEnergia,DatConclusaoACL,PPA
1022,Fora do ACR,2020-07-28,ACL
408,Fora do ACR,2020-03-03,ACL
641,ACR,NaT,ACR
2245,Fora do ACR,2022-05-24,ACL
1908,Fora do ACR,NaT,Nenhum
1006,ACR,2022-01-25,Ambos
170,ACR,NaT,ACR
267,Fora do ACR,NaT,Nenhum
1556,Fora do ACR,NaT,Nenhum
2998,Fora do ACR,NaT,Nenhum


In [196]:
tabelcriterio = definicoesmonitoramento.loc[definicoesmonitoramento.IdcUsinaMonitorada == "Sim"].copy()

In [197]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

tabelcriterio['criterio'] = np.select(
    [
        (tabelcriterio.IdcSituacaoObra == ea),
        (( tabelcriterio.IdcSituacaoObra == na)  &  (tabelcriterio.PPA != nenhum) ),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso != ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso != ok)),
        (tabelcriterio.IdcSituacaoObra == paralisada),
        ((tabelcriterio.DthEnvio.isna()) & (tabelcriterio.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


tabelcriterio['dscriterio'] = np.select(
    [
        (tabelcriterio.IdcSituacaoObra == ea),
        (( tabelcriterio.IdcSituacaoObra == na)  &  (tabelcriterio.PPA != nenhum) ),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso != ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso != ok)),
        (tabelcriterio.IdcSituacaoObra == paralisada),
        ((tabelcriterio.DthEnvio.isna()) & (tabelcriterio.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(tabelcriterio[['IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
76,Em andamento,ACL,LO,OK,0,Usina em Obras em andamento
2870,Não Iniciada,Nenhum,LI,Verificar,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
828,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2299,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
379,Em andamento,ACR,LO,OK,0,Usina em Obras em andamento
760,Em andamento,Nenhum,LO,OK,0,Usina em Obras em andamento
2142,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
1227,Em andamento,ACL,LI,Sem Uso,0,Usina em Obras em andamento
2274,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
1218,Não Iniciada,ACL,LI,OK,1,Sem obras e PPA Ok


In [216]:
slp = "Sem LP"
sim = "Sim"
tabelcriterio['criterio_novo'] = np.select(
    [
        (tabelcriterio.IdcSemPrevisao == sim),
        (tabelcriterio.IdcSituacaoObra == ea),
        (( tabelcriterio.IdcSituacaoObra == na)  &  (tabelcriterio.PPA != nenhum) ),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso != ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso != ok)),
        (tabelcriterio.IdcSituacaoObra == paralisada),
        (tabelcriterio.CondicaoAmbiental == "Sem LP"),
        ((tabelcriterio.DthEnvio.isna()) & (tabelcriterio.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(tabelcriterio[['IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(10))

,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
550,Em andamento,ACR,LI,OK,0,Usina em Obras em andamento
593,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
1453,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1732,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1899,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
411,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
915,Em andamento,ACL,LI,OK,0,Usina em Obras em andamento
1260,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1942,Não Iniciada,Nenhum,LP,Sem Uso,9,"Sem obras, sem PPA e LP OK e Cust não Ok"
1289,Não Iniciada,ACL,LI,OK,1,Sem obras e PPA Ok


In [217]:
#sas = pd.read_clipboard() 
merge = pd.merge(tabelcriterio[['IdeUsinaOutorga','IdcSemPrevisao','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','criterio_novo']],
sas,
how="left",
on= "IdeUsinaOutorga",suffixes=["","_sas"],validate="one_to_one")

In [218]:
merge[merge.criterio_novo != merge['criterio novo']]

,IdeUsinaOutorga,IdcSemPrevisao,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,criterio_novo,criterio_sas,criterio novo,dscriterio
106,32196,Não,Não Iniciada,Nenhum,Sem LP,Não informado,7,6,7,7,"Sem obras, sem PPA e sem Rapeel"
107,32201,Não,Não Iniciada,Nenhum,Sem LP,Não informado,7,6,7,7,"Sem obras, sem PPA e sem Rapeel"
109,32236,Não,Não Iniciada,Nenhum,Sem LP,Não informado,7,6,7,7,"Sem obras, sem PPA e sem Rapeel"
110,32237,Não,Não Iniciada,Nenhum,Sem LP,Não informado,7,6,7,7,"Sem obras, sem PPA e sem Rapeel"
142,32588,Não,Não Iniciada,Nenhum,Sem LP,Não informado,7,6,7,7,"Sem obras, sem PPA e sem Rapeel"
...,...,...,...,...,...,...,...,...,...,...,...
2392,70558,Não,Não Iniciada,Nenhum,Sem LP,Não informado,7,6,7,7,"Sem obras, sem PPA e sem Rapeel"
2393,70559,Não,Não Iniciada,Nenhum,Sem LP,Não informado,7,6,7,7,"Sem obras, sem PPA e sem Rapeel"
2394,70560,Não,Não Iniciada,Nenhum,Sem LP,Não informado,7,6,7,7,"Sem obras, sem PPA e sem Rapeel"
2395,70561,Não,Não Iniciada,Nenhum,Sem LP,Não informado,7,6,7,7,"Sem obras, sem PPA e sem Rapeel"
